# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# common imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

In [ ]:
# System content and user question

# The assistant is useful and kind, and should always answer in Italian, no matter the language used in the question
system_content = "Sei un assistente utile e gentile e, a prescindere dalla lingua con cui ti vengono poste le domande, rispondi sempre in italiano"

# The question the user is asking to the assistant
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Get gpt-4o-mini on openAI to answer, with streaming

from openai import OpenAI

# load api key from .env
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

# Constants
MODEL_GPT = 'gpt-4o-mini'

openai = OpenAI()

openai_messages=[
            {"role":"system", "content":system_content},
            {"role":"user", "content":question}
        ]

def display_stream_openai(stream):
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

response = openai.chat.completions.create(
        model="gpt-4o-mini", 
        messages=openai_messages,
        stream=True
    )

display_stream_openai(response)


In [ ]:
# Get Gemma3 on Ollama to answer

import ollama

MODEL = "gemma3:12b"

!ollama pull gemma3:12b

def display_stream_ollama(stream):
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk['message']['content'] or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

ollama_messages=[
            {"role" : "user", "content" : system_content},
            {"role" : "user", "content" : question}
        ]

response = ollama.chat(
    model=MODEL,
    messages=ollama_messages,
    stream=True
)

display_stream_ollama(response)
